In [4]:
import pandas as pd
import pickle

In [8]:
url_moedas = "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/Moedas?%24format=json"

df_moedas = pd.json_normalize(pd.read_json(url_moedas)["value"])

def cotaMoeda():
    lista = []
    entradas = []
    print("Esta é a lista de moedas dispoíveis:")
    print(df_moedas)
    
    fim = False
    while not fim:
        moeda = input('Insira o símbolo de uma das moedas apresentadas: ')
        if moedas in df_moedas['simbolo']:
            resposta = input('Escolher mais uma moeda? (N para finalizar/qualquer tecla para adicionar) ')
            lista.append(moeda)
            if resposta == "N":
                fim = True
        else:
            print("Moeda inválida")
    
    inicio = input('Insira a data inicial (mm-dd-aaaa): ')
    final = input('Insira a data final (mm-dd-aaaa): ')

    nomes_moedas = "" # variável para nomear o arquivo exportado
    for i in lista:
        url = 'https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoMoedaPeriodo(moeda=@moeda,dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?%40moeda=%27{}%27&%40dataInicial=%27{}%27&%40dataFinalCotacao=%27{}%27&%24format=json'.format(i, inicio, final)
        entrada = pd.json_normalize(pd.read_json(url)['value'])
        entradas.append(entrada)
        nomes_moedas += "_" + i
    df_final = entradas[0]
    
    if len(entradas) > 1:
        for k in range(1,len(entradas)):
            df_final = df_final.merge(entradas[k], on=["dataHoraCotacao", "tipoBoletim"], suffixes=["_" + i for i in lista])
            #  reordenar para as colunas comuns ficarem em primeiro lugar:
            df_final = df_final.reindex(columns=(['dataHoraCotacao','tipoBoletim'] + list([a for a in df_final.columns if a != 'dataHoraCotacao' and a != 'tipoBoletim']) ))
    
    pickle.dump(df_final, open("cotacoes_{}_{}{}.pickle".format(inicio, final, nomes_moedas), "wb"))
    return df_final